In [11]:
import numpy as np
import pandas as pd
import os
import numpy as np
import pandas as pd
import os
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
import time
from sklearn.preprocessing import StandardScaler,MinMaxScaler




In [3]:
start = time.time()


dataset= pd.read_csv("../../Data/data-2.csv")
dataset

,docno,doclen,text
0,LA122989-0001,50,the citi compton ask peopl donat book given hi...
1,LA122989-0002,400,sure traffic pace life gener becom hectic luck...
2,LA122989-0005,32,leona helmsley correct said onli littl peopl p...
3,LA122989-0006,87,your editori dec 14 remark upon result studi i...
4,LA122989-0007,431,soviet presid mikhail s gorbachev day away sev...
...,...,...,...
57509,FT911-3689,16,hundr romanian riot polic arm baton shield sto...
57510,FT911-3690,25,in earli edit financi time april 13 pictur mr ...
57511,FT911-3691,367,few issu quit controversi govern privatis prog...
57512,FT911-3692,375,uk stock suffer anoth sharp revers yesterday f...


In [4]:
%%time
dataset['text_list']= dataset['text'].apply(lambda x: [item for item in str(x).split()])
dataset

CPU times: user 940 ms, sys: 149 ms, total: 1.09 s
Wall time: 1.09 s


,docno,doclen,text,text_list
0,LA122989-0001,50,the citi compton ask peopl donat book given hi...,"[the, citi, compton, ask, peopl, donat, book, ..."
1,LA122989-0002,400,sure traffic pace life gener becom hectic luck...,"[sure, traffic, pace, life, gener, becom, hect..."
2,LA122989-0005,32,leona helmsley correct said onli littl peopl p...,"[leona, helmsley, correct, said, onli, littl, ..."
3,LA122989-0006,87,your editori dec 14 remark upon result studi i...,"[your, editori, dec, 14, remark, upon, result,..."
4,LA122989-0007,431,soviet presid mikhail s gorbachev day away sev...,"[soviet, presid, mikhail, s, gorbachev, day, a..."
...,...,...,...,...
57509,FT911-3689,16,hundr romanian riot polic arm baton shield sto...,"[hundr, romanian, riot, polic, arm, baton, shi..."
57510,FT911-3690,25,in earli edit financi time april 13 pictur mr ...,"[in, earli, edit, financi, time, april, 13, pi..."
57511,FT911-3691,367,few issu quit controversi govern privatis prog...,"[few, issu, quit, controversi, govern, privati..."
57512,FT911-3692,375,uk stock suffer anoth sharp revers yesterday f...,"[uk, stock, suffer, anoth, sharp, revers, yest..."


In [5]:
%%time
model = Word2Vec(sentences=dataset['text_list'], vector_size=100, window=5, min_count=1, workers=4)
model.save("word2vec.model")
model = Word2Vec.load("word2vec.model")
#model.train([["hello", "world"]], total_examples=1, epochs=1)
model.init_sims(replace = True)

CPU times: user 2min, sys: 842 ms, total: 2min
Wall time: 34.2 s


<timed exec>:5: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).


In [6]:
%%time


def vectorize(list_of_docs, model):
    
    features = []

    for tokens in list_of_docs:
        zero_vector = np.zeros(model.vector_size)
        vectors = []
        for token in tokens:
            if token in model.wv:
                try:
                    vectors.append(model.wv[token])
                except KeyError:
                    continue
        if vectors:
            vectors = np.asarray(vectors)
            avg_vec = vectors.mean(axis=0)
            features.append(avg_vec)
        else:
            features.append(zero_vector)
    return features
    
vectorized_docs = vectorize(dataset['text_list'], model=model)
len(vectorized_docs), len(vectorized_docs[0])

CPU times: user 17.1 s, sys: 122 ms, total: 17.3 s
Wall time: 17.3 s


(57514, 100)

In [30]:

dataframe=pd.DataFrame(vectorized_docs)

dataframe['docno']=dataset['docno']
dataframe=dataframe.set_index('docno')
std_scaler = MinMaxScaler()
 
df_scaled = pd.DataFrame(std_scaler.fit_transform(dataframe.to_numpy()),columns=dataframe.columns,index=dataframe.index)
df_scaled
df_scaled.to_csv('Word2Vec.csv',index=True)



In [35]:
word2_vec_features=pd.read_csv("Word2Vec.csv",index_col='docno')

LDA_features=pd.read_csv('reduced_feature_lda.csv')

In [36]:
LDA_features

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.000000,0.000000,0.000000,0.000000,0.328327,0.000000,0.000000,0.045343,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.050496,0.000000,0.180095,0.272665,0.118825,0.000000
1,0.014807,0.019797,0.117585,0.000000,0.075978,0.093444,0.225959,0.026382,0.020667,0.180509,0.040813,0.000000,0.000000,0.000000,0.000000,0.051055,0.000000,0.000000,0.082985,0.039108
2,0.000000,0.000000,0.264091,0.000000,0.000000,0.083620,0.549375,0.000000,0.000000,0.000000,0.000000,0.000000,0.095992,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.047712,0.000000,0.186320,0.000000,0.000000,0.230761,0.000000,0.000000,0.079930,0.000000,0.000000,0.061168,0.000000,0.000000,0.128149,0.058272,0.000000,0.164828,0.041083
4,0.000000,0.000000,0.011777,0.525603,0.000000,0.000000,0.371186,0.000000,0.029616,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.061364,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57509,0.000000,0.000000,0.000000,0.125183,0.000000,0.000000,0.000000,0.000000,0.441163,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.414889,0.000000,0.000000,0.000000,0.000000
57510,0.000000,0.089365,0.000000,0.000000,0.000000,0.154983,0.000000,0.000000,0.000000,0.000000,0.000000,0.157285,0.000000,0.111099,0.000000,0.000000,0.000000,0.474954,0.000000,0.000000
57511,0.000000,0.000000,0.000000,0.259590,0.000000,0.000000,0.137141,0.000000,0.000000,0.000000,0.000000,0.267223,0.000000,0.143540,0.000000,0.000000,0.000000,0.000000,0.000000,0.163430
57512,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.051671,0.000000,0.000000,0.000000,0.000000,0.048055,0.027471,0.835900,0.026644,0.000000,0.000000,0.000000,0.000000,0.000000
